<a href="https://colab.research.google.com/github/VijayaVajjhala/Product-Recommender-Agent/blob/main/Product_Recommendor_and_Comparator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install neo4j

  Using cached neo4j-5.28.1-py3-none-any.whl.metadata (5.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 4.7 MB/s eta 0:00:00


In [8]:
import pandas as pd
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
import json
import hashlib # For generating review_id
from datetime import datetime

In [9]:
NEO4J_URI = "neo4j+s://49db1554.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "em3IQa3dY6_A5fHCg21eRDd138qX0Xxms0xyljCBbQ8"
NEO4J_DATABASE = "neo4j" # Default database name for AuraDB

In [10]:
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    driver.verify_connectivity()
    print("Neo4j connection successful!")
except Exception as e:
    print(f"Neo4j connection failed: {e}")
    print("Please ensure your NEO4J_URI, NEO4J_USERNAME, and NEO4J_PASSWORD are correct.")
    print("If you are running locally, ensure your Neo4j instance is running and accessible.")
    raise

Neo4j connection successful!


In [11]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Embedding model loaded.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded.


In [12]:
from transformers import pipeline

In [13]:
def perform_sentiment_analysis(texts):
    """
    Performs sentiment analysis on a list of texts using a pre-trained
    Hugging Face sentiment analysis model.

    Args:
        texts (list of str): A list of text strings to analyze.

    Returns:
        list of dict: A list of dictionaries, where each dictionary
                      contains the 'label' (e.g., 'POSITIVE', 'NEGATIVE')
                      and 'score' for the sentiment of the corresponding text.
    """
    # Initialize the sentiment-analysis pipeline.
    # The 'distilbert-base-uncased-finetuned-sst-2-english' model is a
    # popular and efficient choice for binary (positive/negative) sentiment.
    # It's fine-tuned on the Stanford Sentiment Treebank (SST-2) dataset.
    print("Loading sentiment analysis model...")
    classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    print("Model loaded.")

    # Perform sentiment analysis
    results = classifier(texts)

    return results


In [20]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
product_metadata_path = '/content/drive/MyDrive/Product Agent/meta_All_Beauty.jsonl.gz'
review_data_path = '/content/drive/MyDrive/Product Agent/All_Beauty.jsonl.gz'

Mounted at /content/drive


In [21]:
def read_jsonl_gz_to_pandas(file_path):
    """
    Reads a .jsonl.gz file into a pandas DataFrame.

    Args:
        file_path (str): The path to the .jsonl.gz file.

    Returns:
        pd.DataFrame: A DataFrame containing the data from the file.
                      Returns None if the file cannot be read.
    """
    try:
        # pd.read_json can directly handle gzipped files and JSON Lines format.
        # 'compression="gzip"' tells pandas to decompress the file.
        # 'lines=True' tells pandas that each line is a separate JSON object.
        df = pd.read_json(file_path, compression='gzip', lines=True)
        print(f"Successfully read {len(df)} records from {file_path}")
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None